In [1]:
#Cargamos el df base del servidor sql.
import pandas as pd
import sqlite3
conn = sqlite3.connect('C:\GitHubRepos\ProyectoFinal\data\processed\Dataset9k.db')
query = "SELECT * FROM control"
df_encuestas = pd.read_sql_query(query, conn)
conn.close()

<>:4: SyntaxWarning: invalid escape sequence '\G'
<>:4: SyntaxWarning: invalid escape sequence '\G'
C:\Users\Josue\AppData\Local\Temp\ipykernel_12712\2811866231.py:4: SyntaxWarning: invalid escape sequence '\G'
  conn = sqlite3.connect('C:\GitHubRepos\ProyectoFinal\data\processed\Dataset9k.db')


In [2]:
#Factorizamos nuestra variable target ()
import json
import os
mapping = {'SocialComunista': 0, 'Socialista': 1, 'Demócrata(Centro)': 2, 'Conservador': 3, 'ExtremaDerecha': 4}
def agrupar_lrscale(valor):
    if 0 <= valor <= 2:
        return 'SocialComunista'
    elif 3 == valor <= 4:
        return 'Socialista'
    elif 5 == valor <= 6:
        return 'Demócrata(Centro)'
    elif 7 == valor <= 8:
        return 'Conservador'
    else:
        return 'ExtremaDerecha'


df_encuestas['ideo'] = df_encuestas['lrscale'].apply(agrupar_lrscale)





df_encuestas['ideo_fc'] = df_encuestas['ideo'].map(mapping)


ruta_archivo_json = "C:/GitHubRepos/ProyectoFinal/data/processed/ideo_factors.json"


with open(ruta_archivo_json, 'w') as f:
    json.dump(mapping, f)
df_encuestas = df_encuestas.apply(lambda col: col.astype(int) if col.dtype == 'float64' else col)
print(df_encuestas.head())

   pplfair  pplhlp  ppltrst  rdtot  tvtot  euftf  gincdif  lrscale  polintr  \
0        5       5        5      0      0      5        3        5        1   
1        4       4        4      0      1      1        3       10        2   
2        4       6        5      0      5      5        1        5        3   
3        6       5        6      1      5      5        2        5        3   
4        5       2        2      2      5      5        2        4        3   

   stfeco  ...  ipadvnta  ipcrtiva  ipeqopta  iplylfra  ipmodsta  ipudrsta  \
0       5  ...         5         2         2         2         2         2   
1       3  ...         5         2         2         2         2         2   
2       2  ...         4         2         1         1         2         2   
3       0  ...         4         4         3         2         2         2   
4       5  ...         2         2         3         3         4         2   

   P_fc  GroupCntry               ideo  ideo_fc  
0     

In [3]:
del df_encuestas['ideo']

In [5]:
df_encuestas.to_csv('C:/GitHubRepos/ProyectoFinal/data/processed/presplit.csv', index=False)

In [6]:
import pandas as pd
import numpy as np, random
from sklearn.model_selection import train_test_split
#Pasamos el filtro de tipo de datos y obligamos a que esten en enteros
random.seed(42)
dtest = pd.read_csv('C:\GitHubRepos\ProyectoFinal\data\processed\presplit.csv')

for col in dtest.select_dtypes(include=['float64']).columns:
    
    try:
        dtest[col] = dtest[col].astype('int64')
    except:
        dtest[col] = pd.to_numeric(dtest[col], downcast='integer')

print(dtest.dtypes)

<>:6: SyntaxWarning: invalid escape sequence '\G'
<>:6: SyntaxWarning: invalid escape sequence '\G'
C:\Users\Josue\AppData\Local\Temp\ipykernel_12712\905134761.py:6: SyntaxWarning: invalid escape sequence '\G'
  dtest = pd.read_csv('C:\GitHubRepos\ProyectoFinal\data\processed\presplit.csv')


pplfair       int64
pplhlp        int64
ppltrst       int64
rdtot         int64
tvtot         int64
euftf         int64
gincdif       int64
lrscale       int64
polintr       int64
stfeco        int64
stfedu        int64
stfgov        int64
stfhlth       int64
stflife       int64
trstplc       int64
trstplt       int64
trstun        int64
imbgeco       int64
aesfdrk       int64
atcherp       int64
happy         int64
health        int64
inprdsc       int64
rlgdgr        int64
sclmeet       int64
ccnthum       int64
impenva       int64
impfreea      int64
impfuna       int64
impricha      int64
impsafea      int64
imptrada      int64
ipadvnta      int64
ipcrtiva      int64
ipeqopta      int64
iplylfra      int64
ipmodsta      int64
ipudrsta      int64
P_fc          int64
GroupCntry    int64
ideo_fc       int64
dtype: object


In [ ]:
#FUNCION SPLIT TRAIN TEST PRINCIPAL.
dtest = pd.read_csv("C:\GitHubRepos\ProyectoFinal\data\processed\presplit.csv")
iX = dtest.drop(['ideo_fc', 'lrscale'], axis=1)
iy = dtest['ideo_fc']




import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
import os

# 1. Carga de datos
dtest = pd.read_csv("C:\GitHubRepos\ProyectoFinal\data\processed\presplit.csv")

# 2. División train/test inicial (CON MUESTREO ESTRATIFICADO GENERAL)
X = dtest.drop(['ideo_fc', 'lrscale', 'cntryfcz', 'origfcz', 'cluster'], axis=1)
y = dtest['ideo_fc']

X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(
    X, y, test_size=0.3,  random_state=42
)#stratify=y,

#Muestreo estratificado general
sampled_data_general = []
n_samples_general = min(4000, len(X_train_full))
X_train_general = X_train_full.sample(n=n_samples_general, random_state=42)
y_train_general = y_train_full.loc[X_train_general.index]

X_train_general, X_test_general, y_train_general, y_test_general = train_test_split(
    X_train_general, y_train_general, test_size=0.3, random_state=42
)#stratify=y_train_general

# 3. Guardado en CSV (opcional)
X_train_general.to_csv('C:/GitHubRepos/ProyectoFinal/data/models/Split/X_train_general.csv', index=False)
X_test_general.to_csv('C:/GitHubRepos/ProyectoFinal/data/models/Split/X_test_general.csv', index=False)
pd.DataFrame(y_train_general).to_csv('C:/GitHubRepos/ProyectoFinal/data/models/Split/y_train_general.csv', index=False)
pd.DataFrame(y_test_general).to_csv('C:/GitHubRepos/ProyectoFinal/data/models/Split/y_test_general.csv', index=False)


# 4. Iteración por clusters (CON MUESTREO ESTRATIFICADO POR CLUSTER)
for cluster in dtest['cluster'].unique():
    # Filtrar *X_train_full e y_train_full* por cluster (USANDO X_train_full)
    X_train_cluster = X_train_full[dtest.loc[X_train_full.index, 'cluster'] == cluster]
    y_train_cluster = y_train_full[dtest.loc[X_train_full.index, 'cluster'] == cluster]

    # Filtrar *X_test_full e y_test_full* por cluster (USANDO X_test_full)
    X_test_cluster = X_test_full[dtest.loc[X_test_full.index, 'cluster'] == cluster]
    y_test_cluster = y_test_full[dtest.loc[X_test_full.index, 'cluster'] == cluster]

    #Muestreo estratificado por cluster
    sampled_data_cluster_train = []
    n_samples_cluster_train = min(700, len(X_train_cluster)) #375 train + 125 test = 500
    X_train_cluster_sampled = X_train_cluster.sample(n=n_samples_cluster_train, random_state=42)
    y_train_cluster_sampled = y_train_cluster.loc[X_train_cluster_sampled.index]

    sampled_data_cluster_test = []
    n_samples_cluster_test = min(300, len(X_test_cluster)) #375 train + 125 test = 500
    X_test_cluster_sampled = X_test_cluster.sample(n=n_samples_cluster_test, random_state=42)
    y_test_cluster_sampled = y_test_cluster.loc[X_test_cluster_sampled.index]

    # 5. Submuestreo y Sobremuestreo (SMOTE después de RUS)
    rus = RandomUnderSampler(random_state=42)
    X_train_resampled, y_train_resampled = rus.fit_resample(X_train_cluster_sampled, y_train_cluster_sampled)

    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_resampled, y_train_resampled)  # ¡CORREGIDO!

    # 7. Creación de DataFrames para cada cluster
    train_cluster_df = pd.DataFrame(X_train_resampled, columns=X_train_cluster.columns)
    train_cluster_df['ideo_fc'] = y_train_resampled

    test_cluster_df = pd.DataFrame(X_test_cluster_sampled, columns = X_test_cluster.columns) #Añadido los nombres de las columnas
    test_cluster_df['ideo_fc'] = y_test_cluster_sampled

    # 8. Guardado en CSVs para cada cluster (SOBREESCRIBIR)
    train_cluster_df.to_csv(
        f'C:/GitHubRepos/ProyectoFinal/data/models/Split/train52C{cluster}.csv', index=False
    )
    test_cluster_df.to_csv(
        f'C:/GitHubRepos/ProyectoFinal/data/models/Split/test52C{cluster}.csv', index=False
    )

    print(f"Archivos CSV para cluster {cluster} creados/sobreescritos exitosamente.")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.feature_selection import SelectKBest, chi2
from xgboost import XGBClassifier  # Import XGBoost
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import joblib
import json
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder  # Import LabelEncoder

# --- Función principal ---
def entrenar_y_evaluar(X_train, y_train, X_test, y_test, target_variable, model_type, ruta_modelo, ruta_configuracion, ruta_dataset):
    resultados = {}
    mejor_modelo = None
    mejor_k = None
    mejores_resultados = None
    mejor_auc = -1  # Inicializamos con el peor valor posible
    mejor_target_variable = None

    for k in range(9, 39):  # Range of k values to test
        # Selección de características
        selector = SelectKBest(score_func=chi2, k=k)
        X_train_new = selector.fit_transform(X_train, y_train)
        X_test_new = selector.transform(X_test)
        mejores_features = X_train.columns[selector.get_support()]

        # --- Tratamiento de clases minoritarias con SMOTE ---
        smote = SMOTE(random_state=42)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train_new, y_train)

        # Optimización de hiperparámetros con RandomizedSearchCV (XGBoost)

        # 1. Label Encode your target variable *before* using it in XGBoost
        le = LabelEncoder()
        y_train_encoded = le.fit_transform(y_train)  # Fit and transform on training data
        y_test_encoded = le.transform(y_test)  # Transform the test using the fitted encoder

        model = XGBClassifier(random_state=42, eval_metric='logloss')  # XGBoost - use_label_encoder removed
        param_grid = {
            'n_estimators': range(50, 301, 50),  # Wider range
            'max_depth': range(3, 11),  # Range for max_depth
            'learning_rate': [0.01, 0.05, 0.1, 0.2],  # Learning rate
            'gamma': [0, 0.1, 0.2],  # Gamma for regularization
            'min_child_weight': range(1, 6),  # Min child weight
            'subsample': [0.6, 0.8, 1.0],  # Subsample ratio
            'colsample_bytree': [0.6, 0.8, 1.0],  # Colsample by tree
            'reg_alpha': [0, 0.1, 1],  # L1 regularization
            'reg_lambda': [0, 0.1, 1]  # L2 regularization
        }
        scoring = 'f1_weighted'
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        X_train_opt = X_train_resampled
        y_train_opt = y_train_resampled

        random_search = RandomizedSearchCV(model, param_grid, n_iter=50, scoring=scoring, cv=cv, n_jobs=-1, random_state=42)  # Increased n_iter
        random_search.fit(X_train_opt, y_train_opt)
        best_model = random_search.best_estimator_

        # Evaluación y almacenamiento de resultados
        y_pred = best_model.predict(X_test_new)  # Predictions will be numerical
        y_pred_decoded = le.inverse_transform(y_pred)  # Decode to original labels
        accuracy = accuracy_score(y_test, y_pred_decoded)  # Compare with original labels
        precision = precision_score(y_test, y_pred_decoded, average='weighted')
        recall = recall_score(y_test, y_pred_decoded, average='weighted')
        f1 = f1_score(y_test, y_pred_decoded, average='weighted')
        auc = roc_auc_score(y_test, best_model.predict_proba(X_test_new), multi_class='ovr')

        resultados[(target_variable, k)] = {
            'mejores_features': mejores_features.tolist(),
            'best_params': random_search.best_params_,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'auc': auc
        }

        print(f"Resultados para {target_variable}, k={k}:")
        print(f"Mejores características: {mejores_features}")
        print(f"Mejores hiperparámetros: {random_search.best_params_}")
        print(f"Exactitud: {accuracy}")
        print(f"Precisión: {precision}")
        print(f"Recall: {recall}")
        print(f"F1-score: {f1}")
        print(f"AUC: {auc}")
        print("-" * 30)

        if auc > mejor_auc:
            mejor_auc = auc
            mejor_modelo = best_model
            mejor_k = k
            mejores_resultados = resultados[(target_variable, k)]
            mejor_target_variable = target_variable

    # Entrenar el modelo final con la mejor configuración (usando TODO el conjunto de entrenamiento)
    selector_final = SelectKBest(score_func=chi2, k=mejor_k)
    X_final = selector_final.fit_transform(X_train, y_train)  # Use X_train and y_train
    rf_final = XGBClassifier(random_state=42, eval_metric='logloss', **mejores_resultados['best_params'])
    rf_final.fit(X_final, le.transform(y_train))  # Encode y_train here as well

    # Guardar el modelo (el MEJOR modelo)
    joblib.dump(rf_final, ruta_modelo)

    # Guardar la configuración (la MEJOR configuración)
    with open(ruta_configuracion, 'w') as f:
        json.dump({'k': mejor_k, 'resultados': mejores_resultados, 'target_variable': mejor_target_variable, 'best_params': mejores_resultados['best_params']}, f)

    # Guardar el dataset final
    df_final = pd.DataFrame(X_final, columns=X_train.columns[selector_final.get_support()])
    df_final[mejor_target_variable] = y_train.loc[df_final.index] #Use y_train
    df_final.to_csv(ruta_dataset, index=False)

    print(f"\nMejor modelo guardado en: {ruta_modelo}")
    print(f"Configuración guardada en: {ruta_configuracion}")
    print(f"Dataset guardado en: {ruta_dataset}")
    
pretrain = pd.read_csv('C:/GitHubRepos/ProyectoFinal/data\models\Split/test52C2.csv',index_col=False)
pretest = pd.read_csv('C:/GitHubRepos/ProyectoFinal/data/models/Split/train52C2.csv',index_col=False)
X_train = pretrain.drop('ideo_fc', axis=1)
y_train = pretrain['ideo_fc'].squeeze()
X_test = pretest.drop('ideo_fc', axis=1)
y_test = pretest['ideo_fc'].squeeze()
'''
X_train = pd.read_csv('C:/GitHubRepos/ProyectoFinal/data/models/Split/X_train_general.csv')
y_train = pd.read_csv('C:/GitHubRepos/ProyectoFinal/data/models/Split/y_train_general.csv')
X_test = pd.read_csv('C:/GitHubRepos/ProyectoFinal/data/models/Split/X_test_general.csv')
y_test = pd.read_csv('C:/GitHubRepos/ProyectoFinal/data/models/Split/y_test_general.csv')
y_train = y_train.squeeze()  # Convierte y_train a Serie
y_test = y_test.squeeze() #Convierte y_test a serie'''
ruta_modelo= 'C:/GitHubRepos/ProyectoFinal/data/models/BRF_ideo_fc.pkl'
ruta_configuracion= 'C:/GitHubRepos/ProyectoFinal/data/models/BRFsetup_ideo_fc.json'
ruta_dataset = 'C:/GitHubRepos/ProyectoFinal/data/models/Datasets/dfRF_ideo_fc.csv'
target_variable = 'ideo_fc'
model_type = RandomForestClassifier
entrenar_y_evaluar(X_train, y_train, X_test, y_test, target_variable, model_type, ruta_modelo, ruta_configuracion, ruta_dataset)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss
import joblib
import json
from imblearn.over_sampling import SMOTE

# --- Función principal para Regresión Logística ---
def entrenar_y_evaluar_regresion_logistica(X_train, y_train, X_test, y_test, target_variable, ruta_modelo, ruta_configuracion, ruta_dataset):
    resultados = {}
    mejor_modelo = None
    mejor_k = None
    mejores_resultados = None
    mejor_auc = -1

    for k in range(9, 39):  # Rango de valores de k a probar
        # Selección de características
        selector = SelectKBest(score_func=chi2, k=k) #o f_classif si hay negativos
        X_train_new = selector.fit_transform(X_train, y_train)
        X_test_new = selector.transform(X_test)
        mejores_features = X_train.columns[selector.get_support()]

        # --- Tratamiento de clases minoritarias con SMOTE ---
        smote = SMOTE(random_state=42)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train_new, y_train)

        # Escalado de características (¡MUY IMPORTANTE para Regresión Logística!)
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_resampled)  # Ajustar y transformar en entrenamiento
        X_test_scaled = scaler.transform(X_test_new)  # Transformar en prueba (usando el scaler ajustado en entrenamiento)


        # Optimización de hiperparámetros con RandomizedSearchCV
        model = LogisticRegression(random_state=42, solver='liblinear', max_iter=10000)  # solver 'liblinear' para penalización L1 y L2
        param_grid = {
            'C': np.logspace(-4, 4, 20),  # Valores de C para regularización
            'penalty': ['l1', 'l2'],  # Penalización L1 o L2
        }
        scoring = 'f1_weighted' #Métrica a optimizar
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        random_search = RandomizedSearchCV(model, param_grid, n_iter=20, scoring=scoring, cv=cv, n_jobs=-1, random_state=42)
        random_search.fit(X_train_scaled, y_train_resampled)
        best_model = random_search.best_estimator_

        # Evaluación y almacenamiento de resultados
        y_pred = best_model.predict(X_test_scaled)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')
        try: #Manejo de error en caso de que la variable objetivo tenga solo una clase
            auc = roc_auc_score(y_test, best_model.predict_proba(X_test_scaled), multi_class='ovr')
        except ValueError:
            auc = 0

        resultados[(target_variable, k)] = {
            'mejores_features': mejores_features.tolist(),
            'best_params': random_search.best_params_,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'auc': auc
        }

        print(f"Resultados para {target_variable}, k={k}:")
        print(f"Mejores características: {mejores_features}")
        print(f"Mejores hiperparámetros: {random_search.best_params_}")
        print(f"Exactitud: {accuracy}")
        print(f"Precisión: {precision}")
        print(f"Recall: {recall}")
        print(f"F1-score: {f1}")
        print(f"AUC: {auc}")
        print("-" * 30)

        if auc > mejor_auc:
            mejor_auc = auc
            mejor_modelo = best_model
            mejor_k = k
            mejores_resultados = resultados[(target_variable, k)]
            mejor_target_variable = target_variable

    # Entrenar el modelo final con la mejor configuración (usando TODO el conjunto de entrenamiento)
    selector_final = SelectKBest(score_func=chi2, k=mejor_k) #o f_classif si hay negativos
    X_final = selector_final.fit_transform(X_train, y_train)
    scaler_final = StandardScaler()
    X_final_scaled = scaler_final.fit_transform(X_final)
    lr_final = LogisticRegression(solver='liblinear', max_iter=10000, **mejores_resultados['best_params'])
    lr_final.fit(X_final_scaled, y_train)

    # Guardar el modelo (el MEJOR modelo)
    joblib.dump(lr_final, ruta_modelo)

    # Guardar la configuración (la MEJOR configuración)
    with open(ruta_configuracion, 'w') as f:
        json.dump({'k': mejor_k, 'resultados': mejores_resultados, 'target_variable': mejor_target_variable, 'best_params': mejores_resultados['best_params']}, f)

    # Guardar el dataset final
    df_final = pd.DataFrame(X_final, columns=X_train.columns[selector_final.get_support()])
    df_final[mejor_target_variable] = y_train.loc[df_final.index] #Use y_train
    df_final.to_csv(ruta_dataset, index=False)

    print(f"\nMejor modelo guardado en: {ruta_modelo}")
    print(f"Configuración guardada en: {ruta_configuracion}")
    print(f"Dataset guardado en: {ruta_dataset}")

# Ejemplo de uso (reemplaza con tus rutas y datos)
pretrain = pd.read_csv('C:/GitHubRepos/ProyectoFinal/data\models\Split/test52C2.csv',index_col=False)
pretest = pd.read_csv('C:/GitHubRepos/ProyectoFinal/data/models/Split/train52C2.csv',index_col=False)
X_train = pretrain.drop('ideo_fc', axis=1)
y_train = pretrain['ideo_fc'].squeeze()
X_test = pretest.drop('ideo_fc', axis=1)
y_test = pretest['ideo_fc'].squeeze()

ruta_modelo = 'C:/GitHubRepos/ProyectoFinal/data/models/LR_ideo_fc.pkl'
ruta_configuracion = 'C:/GitHubRepos/ProyectoFinal/data/models/LRsetup_ideo_fc.json'
ruta_dataset = 'C:/GitHubRepos/ProyectoFinal/data/models/Datasets/dfLR_ideo_fc.csv'
target_variable = 'ideo_fc'

entrenar_y_evaluar_regresion_logistica(X_train, y_train, X_test, y_test, target_variable, ruta_modelo, ruta_configuracion, ruta_dataset)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB  # Modelo Bayesiano Gaussiano
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss
import joblib
import json
from imblearn.over_sampling import SMOTE

# --- Función principal para Naive Bayes Gaussiano ---
def entrenar_y_evaluar_naive_bayes(X_train, y_train, X_test, y_test, target_variable, ruta_modelo, ruta_configuracion, ruta_dataset):
    resultados = {}
    mejor_modelo = None
    mejor_k = None
    mejores_resultados = None
    mejor_auc = -1

    for k in range(9, 39):  # Rango de valores de k a probar
        # Selección de características
        selector = SelectKBest(score_func=chi2, k=k)  # o f_classif si hay negativos
        X_train_new = selector.fit_transform(X_train, y_train)
        X_test_new = selector.transform(X_test)
        mejores_features = X_train.columns[selector.get_support()]

        # --- Tratamiento de clases minoritarias con SMOTE ---
        smote = SMOTE(random_state=42)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train_new, y_train)

        # Escalado de características (Opcional para Naive Bayes, pero a veces ayuda)
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_resampled)
        X_test_scaled = scaler.transform(X_test_new)

        # Entrenamiento del modelo Naive Bayes Gaussiano
        model = GaussianNB()  # No hay hiperparámetros para ajustar en este caso
        model.fit(X_train_scaled, y_train_resampled)

        # Evaluación y almacenamiento de resultados
        y_pred = model.predict(X_test_scaled)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')
        try:
            auc = roc_auc_score(y_test, model.predict_proba(X_test_scaled), multi_class='ovr')
        except ValueError:
            auc = 0

        resultados[(target_variable, k)] = {
            'mejores_features': mejores_features.tolist(),
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'auc': auc
        }

        print(f"Resultados para {target_variable}, k={k}:")
        print(f"Mejores características: {mejores_features}")
        print(f"Exactitud: {accuracy}")
        print(f"Precisión: {precision}")
        print(f"Recall: {recall}")
        print(f"F1-score: {f1}")
        print(f"AUC: {auc}")
        print("-" * 30)

        if auc > mejor_auc:
            mejor_auc = auc
            mejor_modelo = model
            mejor_k = k
            mejores_resultados = resultados[(target_variable, k)]
            mejor_target_variable = target_variable

    # Entrenar el modelo final con la mejor configuración (usando TODO el conjunto de entrenamiento)
    selector_final = SelectKBest(score_func=chi2, k=mejor_k)  # o f_classif si hay negativos
    X_final = selector_final.fit_transform(X_train, y_train)
    scaler_final = StandardScaler()
    X_final_scaled = scaler_final.fit_transform(X_final)
    nb_final = GaussianNB()
    nb_final.fit(X_final_scaled, y_train)

    # Guardar el modelo (el MEJOR modelo)
    joblib.dump(nb_final, ruta_modelo)

    # Guardar la configuración (la MEJOR configuración)
    with open(ruta_configuracion, 'w') as f:
        json.dump({'k': mejor_k, 'resultados': mejores_resultados, 'target_variable': mejor_target_variable}, f)

    # Guardar el dataset final
    df_final = pd.DataFrame(X_final, columns=X_train.columns[selector_final.get_support()])
    df_final[mejor_target_variable] = y_train.loc[df_final.index]  # Use y_train
    df_final.to_csv(ruta_dataset, index=False)

    print(f"\nMejor modelo guardado en: {ruta_modelo}")
    print(f"Configuración guardada en: {ruta_configuracion}")
    print(f"Dataset guardado en: {ruta_dataset}")


# Ejemplo de uso (reemplaza con tus rutas y datos)
pretrain = pd.read_csv('C:/GitHubRepos/ProyectoFinal/data\models\Split/test52C2.csv', index_col=False)
pretest = pd.read_csv('C:/GitHubRepos/ProyectoFinal/data/models/Split/train52C2.csv', index_col=False)
X_train = pretrain.drop('ideo_fc', axis=1)
y_train = pretrain['ideo_fc'].squeeze()
X_test = pretest.drop('ideo_fc', axis=1)
y_test = pretest['ideo_fc'].squeeze()

ruta_modelo = 'C:/GitHubRepos/ProyectoFinal/data/models/NB_ideo_fc.pkl'
ruta_configuracion = 'C:/GitHubRepos/ProyectoFinal/data/models/NBsetup_ideo_fc.json'
ruta_dataset = 'C:/GitHubRepos/ProyectoFinal/data/models/Datasets/dfNB_ideo_fc.csv'
target_variable = 'ideo_fc'

entrenar_y_evaluar_naive_bayes(X_train, y_train, X_test, y_test, target_variable, ruta_modelo, ruta_configuracion, ruta_dataset)

In [80]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.preprocessing import StandardScaler, LabelEncoder
import joblib
import json
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# --- Función principal para Red Neuronal ---
def entrenar_y_evaluar_red_neuronal(X_train, y_train, X_test, y_test, target_variable, ruta_modelo, ruta_configuracion, ruta_dataset):
    resultados = {}
    mejor_modelo = None
    mejor_k = None
    mejores_resultados = None
    mejor_auc = -1

    for k in range(9, 39):  # Rango de valores de k a probar
        # Selección de características
        selector = SelectKBest(score_func=chi2, k=k)
        X_train_new = selector.fit_transform(X_train, y_train)
        X_test_new = selector.transform(X_test)
        mejores_features = X_train.columns[selector.get_support()]

        # --- Tratamiento de clases minoritarias con SMOTE ---
        smote = SMOTE(random_state=42)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train_new, y_train)

        # Escalado de características
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_resampled)
        X_test_scaled = scaler.transform(X_test_new)

        # --- Diseño y compilación del modelo de red neuronal ---
        model = keras.Sequential([
            layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
            layers.Dropout(0.2),  # Capa de dropout para regularización
            layers.Dense(32, activation='relu'),
            layers.Dropout(0.2),
            layers.Dense(len(np.unique(y_train)), activation='softmax')  # Capa de salida con softmax
        ])

        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',  # o 'categorical_crossentropy' si y_train está en one-hot encoding
                      metrics=['accuracy'])

        # Entrenamiento del modelo
        model.fit(X_train_scaled, y_train_resampled, epochs=50, batch_size=32, verbose=0)  # Ajusta epochs y batch_size

        # Evaluación y almacenamiento de resultados
        y_pred = np.argmax(model.predict(X_test_scaled), axis=1)  # Obtener las clases predichas
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')
        try:
            auc = roc_auc_score(y_test, model.predict(X_test_scaled), multi_class='ovr', average='weighted')
        except ValueError:
            auc = 0

        resultados[(target_variable, k)] = {
            'mejores_features': mejores_features.tolist(),
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'auc': auc
        }

        print(f"Resultados para {target_variable}, k={k}:")
        print(f"Mejores características: {mejores_features}")
        print(f"Exactitud: {accuracy}")
        print(f"Precisión: {precision}")
        print(f"Recall: {recall}")
        print(f"F1-score: {f1}")
        print(f"AUC: {auc}")
        print("-" * 30)

        if auc > mejor_auc:
            mejor_auc = auc
            mejor_modelo = model
            mejor_k = k
            mejores_resultados = resultados[(target_variable, k)]
            mejor_target_variable = target_variable
            mejor_scaler = scaler
            mejor_selector = selector

    # Entrenar el modelo final con la mejor configuración (usando TODO el conjunto de entrenamiento)
    X_final = mejor_selector.transform(X_train)
    X_final_scaled = mejor_scaler.transform(X_final)
    nn_final = keras.models.clone_model(mejor_modelo)  # Crea un nuevo modelo con la misma arquitectura
    nn_final.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    nn_final.fit(X_final_scaled, y_train, epochs=50, batch_size=32, verbose=0)  # Reentrena con todos los datos

    # Guardar el modelo (el MEJOR modelo)
    nn_final.save(ruta_modelo)  # Guarda en formato h5

    # Guardar la configuración (la MEJOR configuración)
    with open(ruta_configuracion, 'w') as f:
        json.dump({'k': mejor_k, 'resultados': mejores_resultados, 'target_variable': mejor_target_variable}, f)

    # Guardar el dataset final
    df_final = pd.DataFrame(X_final, columns=X_train.columns[mejor_selector.get_support()])
    df_final[mejor_target_variable] = y_train.loc[df_final.index]  # Use y_train
    df_final.to_csv(ruta_dataset, index=False)

    print(f"\nMejor modelo guardado en: {ruta_modelo}")
    print(f"Configuración guardada en: {ruta_configuracion}")
    print(f"Dataset guardado en: {ruta_dataset}")

# Ejemplo de uso (reemplaza con tus rutas y datos)
# ... (tu código de carga de datos y definición de rutas)

entrenar_y_evaluar_red_neuronal(X_train, y_train, X_test, y_test, target_variable, ruta_modelo, ruta_configuracion, ruta_dataset)

ModuleNotFoundError: No module named 'tensorflow'